In [13]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import sklearn

In [14]:

account_data = pd.read_csv('data/account.csv', sep=";")
card_data = pd.read_csv('data/card_dev.csv', sep=";")
client_data = pd.read_csv('data/client.csv', sep=";")
disp_data = pd.read_csv('data/disp.csv', sep=";")
district_data = pd.read_csv('data/district.csv', sep=";")
loan_data = pd.read_csv('data/loan_dev.csv', sep=";")
trans_data = pd.read_csv('data/trans_dev.csv', sep=";")



C:\Users\Francisco\AppData\Local\Temp\ipykernel_3888\2549921809.py:7: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_data = pd.read_csv('data/trans_dev.csv', sep=";")


In [15]:
# account_data.median()
account_data.describe()


,account_id,district_id,date
count,4500.000000,4500.000000,4500.000000
mean,2786.067556,37.310444,951654.608667
std,2313.811984,25.177217,14842.188377
min,1.000000,1.000000,930101.000000
25%,1182.750000,13.000000,931227.000000
50%,2368.000000,38.000000,960102.000000
75%,3552.250000,60.000000,961101.000000
max,11382.000000,77.000000,971229.000000


In [16]:
card_data.describe()

,card_id,disp_id,issued
count,177.000000,177.000000,177.000000
mean,433.576271,3031.723164,954971.259887
std,290.507562,2632.338944,7217.974691
min,3.000000,41.000000,931107.000000
25%,183.000000,1080.000000,950616.000000
50%,397.000000,2513.000000,960221.000000
75%,661.000000,4270.000000,960831.000000
max,1247.000000,13660.000000,961231.000000


(mean, mode and median) and dispersion: range , variance, maximum, minimum, quartiles (including the interquartile range), and standard deviation.

In [17]:
from functools import partial
q_25 = partial(pd.Series.quantile, q=0.25)
q_25.__name__ = "25%"
q_75 = partial(pd.Series.quantile, q=0.75)
q_75.__name__ = "75%"

In [18]:
def statFunc(x):
    stats = x.agg(["mean", "var", "std", "min", q_25, "median", q_75, "max"])
    IQ = (stats.loc["75%"] - stats.loc["25%"])
    stats.loc["lower_limit"] = stats.loc["25%"] - 1.5*IQ
    stats.loc["upper_limit"] = stats.loc["75%"] + 1.5*IQ
    stats.loc["outliers-"] = x[x<stats.loc["lower_limit"]].count()
    stats.loc["outliers+"] = x[x>stats.loc["upper_limit"]].count()
    return stats


In [19]:
stats = statFunc(loan_data[['amount', 'duration']])

In [20]:
stats

,amount,duration
mean,1.453086e+05,35.853659
var,1.107700e+10,280.051913
std,1.052473e+05,16.734752
min,4.980000e+03,12.000000
25%,6.832800e+04,24.000000
median,1.148040e+05,36.000000
75%,1.986000e+05,48.000000
max,5.385000e+05,60.000000
lower_limit,-1.270800e+05,-12.000000
upper_limit,3.940080e+05,84.000000


In [21]:
loan_data.isna().sum()

loan_id       0
account_id    0
date          0
amount        0
duration      0
payments      0
status        0
dtype: int64

In [28]:
statFunc(district_data[['no. of inhabitants', 'no. of cities ', 'average salary ']])
# district_data.columns

,no. of inhabitants,no. of cities,average salary
mean,1.338849e+05,6.259740,9031.675325
var,1.874530e+10,5.931647,624419.748462
std,1.369135e+05,2.435497,790.202347
min,4.282100e+04,1.000000,8110.000000
25%,8.585200e+04,5.000000,8512.000000
median,1.088710e+05,6.000000,8814.000000
75%,1.390120e+05,8.000000,9317.000000
max,1.204953e+06,11.000000,12541.000000
lower_limit,6.112000e+03,0.500000,7304.500000
upper_limit,2.187520e+05,12.500000,10524.500000


In [29]:
statFunc(card_data)


C:\Users\Francisco\AppData\Local\Temp\ipykernel_3888\1061797806.py:2: FutureWarning: ['type'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  stats = x.agg(["mean", "var", "std", "min", q_25, "median", q_75, "max"])


,card_id,disp_id,type,issued
mean,433.576271,3.031723e+03,NaN,9.549713e+05
var,84394.643297,6.929208e+06,NaN,5.209916e+07
std,290.507562,2.632339e+03,NaN,7.217975e+03
min,3.000000,4.100000e+01,classic,9.311070e+05
25%,183.000000,1.080000e+03,NaN,9.506160e+05
median,397.000000,2.513000e+03,NaN,9.602210e+05
75%,661.000000,4.270000e+03,NaN,9.608310e+05
max,1247.000000,1.366000e+04,junior,9.612310e+05
lower_limit,-534.000000,-3.705000e+03,NaN,9.352935e+05
upper_limit,1378.000000,9.055000e+03,NaN,9.761535e+05


In [34]:
statFunc(card_data['issued'])

mean           9.549713e+05
var            5.209916e+07
std            7.217975e+03
min            9.311070e+05
25%            9.506160e+05
median         9.602210e+05
75%            9.608310e+05
max            9.612310e+05
lower_limit    9.352935e+05
upper_limit    9.761535e+05
outliers-      1.000000e+00
outliers+      0.000000e+00
Name: issued, dtype: float64